# Data Download
Date Created: 2023-10-31

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [2]:
import geemap

## [Dynamic World V1](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1)

In [5]:
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2021-04-02')
END = START.advance(100, 'day')

col_filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(121.0437, 14.6760)),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
m = geemap.Map()
m.set_center(121.0437, 14.6760, 12)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

Map(center=[14.676, 121.0437], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## [Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD)

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [3]:
import geemap

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import pandas as pd
import IPython.display as disp
import json
import csv 
import os
import datetime
import requests
from datetime import datetime
from datetime import timedelta
import time
%matplotlib inline

In [3]:
geoJSON_path = "data\\raw\\Sentinel1\\geojson"
output_path = "r'data\\raw\\Sentinel1\\images"

In [4]:
#create function to download the image or imagecollection as you desire
def downloader(ee_object,region): 
    try:
        #download image
        if isinstance(ee_object, ee.image.Image):
            print('Its Image')
            url = ee_object.getDownloadUrl({
                    'scale': 10,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
        
        #download imagecollection
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            print('Its ImageCollection')
            ee_object_new = ee_object.mosaic()
            url = ee_object_new.getDownloadUrl({
                    'scale': 10,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
    except:
        print("Could not download")

In [6]:
def getDATES(aoi, date_event):
    
    n = 175
    output = []
    for i in range(1, n+1):
        output.append(i)
    multiple = 120 #1/3 of a year
    
    for iend in range(1,100):
        # print(iend*multiple)
        im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                    .filterBounds(aoi)
                    .filterDate(ee.Date(date_event+timedelta(days=1)),ee.Date(date_event+timedelta(days=iend*multiple)))
                    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                    .filter(ee.Filter.inList('relativeOrbitNumber_start', output))
                    .sort('system:time_start'))
        acq_times_end = im_coll.aggregate_array('system:time_start').getInfo()
        orbitN = im_coll.aggregate_array('relativeOrbitNumber_start').getInfo()
        if len(acq_times_end) > 1:
            endDATE = datetime.fromtimestamp(mktime(time.gmtime(acq_times_end[0]/1000)))
            orbitchosen = orbitN[0]
            break
    
    for istart in range(1,100):
        # print(istart*multiple)
        im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                    .filterBounds(aoi)
                    .filterDate(ee.Date(date_event-timedelta(days=istart*multiple)),ee.Date(date_event-timedelta(days=1)))
                    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                    .filter(ee.Filter.eq('relativeOrbitNumber_start', orbitchosen))
                    .sort('system:time_start'))
        acq_times_start = im_coll.aggregate_array('system:time_start').getInfo()
        if len(acq_times_start) > 1:
            startDATE = datetime.fromtimestamp(mktime(time.gmtime(acq_times_start[-1]/1000)))
            break

    return startDATE, endDATE, orbitchosen

In [7]:
def download_url(path, newfile):
    t0 = time.time()
    url = path
    fn = newfile
    try:
        r = requests.get(url)
        with open(fn, 'wb') as f:
            f.write(r.content)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)

In [ ]:
t0 = time.time()
from datetime import datetime
from time import mktime
for filename in os.listdir(geoJSON_path):
    json_fid = int(filename[:-9])
    if json_fid in df_sample_fid.values:
        try:
            print(filename)
            # load geoJSON file
            with open(geoJSON_path+'\\'+filename) as fa:
                geoJSON = json.load(fa)
                
            coords = geoJSON['features'][0]['geometry']['coordinates']
            aoi = ee.Geometry.Polygon(coords)
            
        
            idx = int(np.where(date_info["fid2"] == json_fid)[0])
            date_event = datetime(date_info["YYYY"][idx], date_info["MM"][idx], date_info["DD"][idx])
            startDATE, endDATE, orbitchosen = getDATES(aoi, date_event)
            
            im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                        .filterBounds(aoi)
                        .filterDate(ee.Date(startDATE),ee.Date(endDATE+timedelta(days=1)))
                        .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                        .filter(ee.Filter.eq('relativeOrbitNumber_start', orbitchosen))
                        .sort('system:time_start')) 
            
            im_list = im_coll.toList(im_coll.size())
            acq_times = im_coll.aggregate_array('system:time_start').getInfo()
            
            im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi)
            im2 = ee.Image(im_list.get(-1)).select('VV').clip(aoi)
            ratio = im1.divide(im2)

            region = aoi.toGeoJSONString()#region must in JSON format
            path = downloader(ratio,region)#call function
            newfile = str('ratio_fid2_'+str(json_fid)+'_'+'orbit'+str(orbitchosen)+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[0]/1000))).replace("/","")+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[-1]/1000))).replace("/","")+".zip")
            t0 = time.time()
            result = download_url(path, newfile)
            print('url:', result[0], 'time:', result[1])
            
            path = downloader(im1,region)#call function
            newfile = str('im1_fid2_'+str(json_fid)+'_'+'orbit'+str(orbitchosen)+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[0]/1000))).replace("/","")+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[-1]/1000))).replace("/","")+".zip")
            t0 = time.time()
            result = download_url(path, newfile)
            print('url:', result[0], 'time:', result[1])
            
            path = downloader(im2,region)#call function
            newfile = str('im2_fid2_'+str(json_fid)+'_'+'orbit'+str(orbitchosen)+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[0]/1000))).replace("/","")+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[-1]/1000))).replace("/","")+".zip")
            t0 = time.time()
            result = download_url(path, newfile)
            print('url:', result[0], 'time:', result[1])
        except:
            pass
print('Total time:', time.time() - t0)